# PyVista - NASA P3 Case Directory Read & Analysis

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pyvista as pv

## Define Helper Functions to Slice the Internal Mesh

In [2]:
def slice_center(mesh, 
                 NORMAL='z', 
                 ORIGIN=[0, 0, 0], 
                 translate=False, 
                 CONTOUR=False
                ):
    """Slice mesh through center in normal direction, move to zero normal."""
    slice_mesh = mesh.slice(normal=NORMAL, origin=ORIGIN, contour=CONTOUR)
    if translate is not False:
        if NORMAL == 'z':
            slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
        elif NORMAL == 'x':
            slice_mesh.translate((-slice.mesh.center[0], 0, 0), inplace=True)
        elif NORMAL == 'y':
            slice_mesh.translate((0, -slice.mesh.center[1], 0), inplace=True)
        else:
            print('ERROR: Normal not found - ', NORMAL)
            slice_mesh = None
    return slice_mesh

In [3]:
def slice_multiple(mesh, NORMAL='z', nslices=2, CONTOUR=False):
    """Slice mesh multiple times along specified axis, move to z=0."""
    slice_mesh = mesh.slice_along_axis(n=nslices, axis=NORMAL, contour=CONTOUR)
    #slice_mesh.translate((0, 0, -slice_mesh.center[-1]), inplace=True)
    return slice_mesh

## Define the Case Directory

In [4]:
case_directory = '/Users/jrobrien/Dissertation/data/solutions/NASA_noPylons_v2_tas120_aoa0_900T33/'

In [5]:
reader = pv.POpenFOAMReader(case_directory + 'NASA_noPylons_v2_tas120_aoa0_900T33.OpenFOAM')

## Subset the final run 

In [6]:
reader.time_values[-1]

786.0

In [7]:
%%time
reader.set_active_time_value(reader.time_values[-1])
reader.cell_to_point_creation = True
mesh = reader.read()

CPU times: user 4.59 s, sys: 732 ms, total: 5.32 s
Wall time: 5.3 s


In [8]:
internal_mesh = mesh["internalMesh"]
boundaries = mesh["boundary"]

In [9]:
boundaries['inlet']['U'][0][0]

120.0

In [10]:
internal_mesh

UnstructuredGrid (0x1571d3b80)
  N Cells:    10691736
  N Points:   11526824
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   -2.000e+02, 2.000e+02
  N Arrays:   18

## Slice Mesh to View Aircraft - Plot

### Z-slice - move origin

In [11]:
## scale velocities based on input patch velocities
internal_mesh['U'] = internal_mesh['U'] / boundaries['inlet']['U'][0][0]

In [12]:
internal_mesh

UnstructuredGrid (0x1571d3b80)
  N Cells:    10691736
  N Points:   11526824
  X Bounds:   -2.000e+02, 2.000e+02
  Y Bounds:   -2.000e+02, 2.000e+02
  Z Bounds:   -2.000e+02, 2.000e+02
  N Arrays:   18

In [14]:
slice_mesh2 = slice_center(internal_mesh, NORMAL='z', CONTOUR=False)
pt = pv.Plotter()
pt.add_mesh(slice_mesh2, scalars='U-normed', preference='cell', cmap='bwr')
pt.show(cpos="xy")

2023-05-23 22:51:10.048 ( 160.268s) [           40B0D]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-05-23 22:51:10.134 ( 160.354s) [           40B0D]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


Widget(value="<iframe src='http://localhost:57182/index.html?ui=P_0x157228400_1&reconnect=auto' style='width: …

### Ghosting

In [20]:
slice_mesh2['U']

pyvista_ndarray([[ 9.9998313e-01, -1.2729675e-04, -1.5279350e-07],
                 [ 9.9998289e-01, -1.2968862e-04, -1.5696264e-07],
                 [ 9.9998271e-01, -1.3209645e-04, -1.6117146e-07],
                 ...,
                 [ 1.0000080e+00, -7.9970857e-07, -2.8634107e-08],
                 [ 1.0000085e+00, -9.3770080e-07, -3.0634460e-08],
                 [ 1.0000089e+00, -1.0880872e-06, -3.2752350e-08]],
                dtype=float32)

In [30]:
abs(120 - internal_mesh['U']) == 120

pyvista_ndarray([[False,  True,  True],
                 [False,  True,  True],
                 [False,  True,  True],
                 ...,
                 [False, False, False],
                 [False, False, False],
                 [False, False, False]])

In [25]:
np.argwhere(abs(120 - slice_mesh2['U']) == 120)

array([[    0,     2],
       [    1,     2],
       [    2,     2],
       ...,
       [56459,     2],
       [56460,     1],
       [56460,     2]])

In [28]:
## try out `ghosting` to remove all freestream values
ghosts = np.argwhere(abs(120 - internal_mesh["U"]) == 120.0)

# This will act on the mesh inplace to mark those cell indices as ghosts
internal_mesh.remove_cells(ghosts)
ghosts

array([[    0,     2],
       [    1,     2],
       [    2,     2],
       ...,
       [56459,     2],
       [56460,     1],
       [56460,     2]])

In [ ]:
#slice_mesh2['U'] = slice_mesh2['U'] / boundaries['inlet']['U'][0][0]

In [31]:
slice_mesh3 = slice_center(internal_mesh, NORMAL='z', CONTOUR=False)
pt = pv.Plotter()
pt.add_mesh(slice_mesh3, scalars='U', preference='cell')
pt.show(cpos="xy")

2023-05-23 22:59:03.548 ( 633.757s) [           40B0D]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.
2023-05-23 22:59:03.630 ( 633.839s) [           40B0D]      vtkPolyhedron.cxx:1742  WARN| A cell with a non-manifold triangulation has been encountered. This cell cannot be contoured.


Widget(value="<iframe src='http://localhost:57182/index.html?ui=P_0x3d7835ac0_3&reconnect=auto' style='width: …

### y-axis slice

In [ ]:
slice3 = slice_center(mesh['internalMesh'], NORMAL='y')

In [ ]:
slice3 = slice_center(mesh['internalMesh'], NORMAL='y', ORIGIN=[0, 0, 5])
pt = pv.Plotter()
pt.add_mesh(slice3, scalars='U')
pt.show(cpos="zx")

### x-slice

In [ ]:
slice4 = slice_center(mesh['internalMesh'], NORMAL='x', ORIGIN=[15, 0, 0], CONTOUR=True)
pt = pv.Plotter()
pt.add_mesh(slice4, scalars='U', cmap='Blues')
pt.show(cpos="zy")

### Polyslice

In [ ]:
sliceA = slice_multiple(mesh['internalMesh'], NORMAL='z', nslices=10, CONTOUR=True)
sliceB = slice_multiple(mesh['internalMesh'], NORMAL='x', nslices=10, CONTOUR=True)
sliceB = slice_multiple(mesh['internalMesh'], NORMAL='y', nslices=10, CONTOUR=True)
pt = pv.Plotter()
pt.add_mesh(sliceA + sliceB + sliceC, scalars='U')
pt.show(cpos="xy")

## Thresholding - Not working

In [ ]:
threshed = mesh['internalMesh'].threshold(value=120)